# MUWCLASS CSCv2 TD construction pipeline 

### Contact huiyang@gwu.edu if you have any questions
### see our paper at https://arxiv.org/abs/2206.13656
#### It is mostly related to Section 2 in the paper

* It is noted that you may get a slightly different TD using a different computer or different versions of python or python packages.
* It may takes a couple of hours to built the TD when first time running it, particurlay it is time-cunsuming to run the cross-matching with multiwavelngth catalogs

This notebook is available at https://github.com/huiyang-astro/MUWCLASS_CSCv2 

* This notebook was run in CIAO 4.14 with Python 3.9 
* run the follow code to create a new conda environment ciao-4.14-muwclass; if you already have ciao-4.14 installed with Python 3.9, you can use your own conda environment with additional Python packages installed from below
* conda create -n ciao-4.14-muwclass -c https://cxc.cfa.harvard.edu/conda/ciao -c conda-forge ciao sherpa ds9 ciao-contrib caldb_main marx python=3.9


* run 'bash install-packages.sh' under ciao-4.14-muwclass environment to install all required packages 

* then, make sure to enable widgetsnbextension and ipyaladin, run
* jupyter nbextension enable --py widgetsnbextension
* jupyter nbextension enable --py --sys-prefix ipyaladin on your terminal

* You might also need to manually register the existing ds9 with the xapns name server by selecting the ds9 File->XPA->Connect menu option so your ds9 will be fully accessible to pyds9.


In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from astropy.io.fits import getdata
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier
from astropy.table import Table
from astroquery.xmatch import XMatch
import time

import sys  
sys.path.insert(0, '../')

from prepare_library import atnf_pos, create_perobs_data, cal_ave, add_MW, confusion_clean, TD_clean
from muwclass_library import prepare_cols

import warnings
warnings.filterwarnings('ignore')

Vizier.ROW_LIMIT = -1
exnum = -999999.

In [3]:
# define some directories and output name

data_dir = './data'
field_name = 'CSCv2_TD_0823'
verb = 0

query_dir = '../demo/data/query'

# YSOs 
### from multiple molecular clouds and open clusters (Megeath et al. 2012; Povich et al. 2011; Ozawa et al. 2005; Giardino et al. 2007; Rebull et al. 2011; Delgado et al. 2011);

In [6]:
YSO1 = Vizier(catalog="J/AJ/144/192",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2012AJ....144..192M
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(Cl='=|P|D')[0]
YSO1 = YSO1['_RAJ2000','_DEJ2000','Cl'].to_pandas().rename(columns={'Cl':'SubClass'})
YSO1['e_Pos'], YSO1['ref'] = np.nan, '2012AJ....144..192M'
print(len(YSO1),Counter(YSO1['SubClass']))

YSO2 = Vizier(catalog="J/ApJS/194/14/catalog",row_limit=-1,#https://ui.adsabs.harvard.edu/?#abs/2011ApJS..194...14P
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(Stage='!=A')[0]
YSO2 = YSO2['_RAJ2000','_DEJ2000','Stage'].to_pandas().rename(columns={'Stage':'SubClass'})
YSO2['e_Pos'], YSO2['ref'] = np.nan, '2011ApJS..194...14P'
print(len(YSO2),Counter(YSO2['SubClass']))


YSO3 = Vizier(catalog="J/A+A/429/963",row_limit=-1,#https://ui.adsabs.harvard.edu/?#abs/2005A%26A...429..963O
    columns=['*', '_RAJ2000', '_DEJ2000','e_Pos']).query_constraints(Class='!=nIII')[0]
YSO3 = YSO3['_RAJ2000','_DEJ2000','e_Pos','Class'].to_pandas().rename(columns={'Class':'SubClass'})
YSO3['ref'] = '2005A&A...429..963O'
print(len(YSO3),Counter(YSO3['SubClass']))


YSO4 = Vizier(catalog="J/A%2bA/463/275/table5",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2007A%26A...463..275G
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(clYSO='=|I|I/II|II|II/III|III')[0]
YSO4 = YSO4['_RAJ2000','_DEJ2000','clYSO'].to_pandas().rename(columns={'clYSO':'SubClass'})
YSO4['e_Pos'], YSO4['ref'] = np.nan, '2007A&A...463..275G'
print(len(YSO4),Counter(YSO4['SubClass']))

YSO5 = Vizier(catalog="J/ApJS/196/4",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2011ApJS..196....4R
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(St='=|k|n')[0]
YSO5 = YSO5['_RAJ2000','_DEJ2000','St'].to_pandas().rename(columns={'St':'SubClass'})
YSO5['e_Pos'], YSO5['ref'] = np.nan,  '2011ApJS..196....4R'
print(len(YSO5),Counter(YSO5['SubClass']))

YSO6 = Vizier(catalog="J/A%2bA/531/A141/catalog",row_limit=-1,#https://ui.adsabs.harvard.edu/?#abs/2011A%26A...531A.141D
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(MmD='=2')[0]
YSO6 = YSO6['_RAJ2000','_DEJ2000','MmD'].to_pandas().rename(columns={'MmD':'SubClass'})
YSO6['e_Pos'], YSO6['ref'] = np.nan, '2011A&A...531A.141D'
print(len(YSO6),Counter(YSO6['SubClass']))



3419 Counter({'D': 2991, 'P': 428})
808 Counter({'II': 478, '0/I': 247, 'III': 83})
72 Counter({'II': 26, '': 22, 'III': 17, 'I': 7})
56 Counter({'II': 20, 'III': 16, 'I/II': 9, 'I': 9, 'II/III': 2})
272 Counter({'k': 178, 'n': 94})
308 Counter({2: 308})


In [7]:
df_YSOs = pd.concat([YSO1, YSO2, YSO3, YSO4, YSO5, YSO6])
df_YSOs = df_YSOs.reset_index(drop=True)
df_YSOs['Class']='YSO'
print(len(df_YSOs),'YSOs')

4935 YSOs


# STARs
### from the Catalog of Stellar Spectral Classifications (Skiff 2014) with O, B or W (e.g., WN, WR stars) types are labeled as HM-STARs and A, F, G, K, or M types are labeled as LM-STARs;

In [8]:
stars = Vizier(catalog="B/mk/mktypes",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2014yCat....1.2023S
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(Mag='<=23')[0]# Fainter sources with Mag > 23 were removed
stars = stars['_RAJ2000','_DEJ2000','Name','SpType','Bibcode','Remarks','Mag'].to_pandas()
stars = stars.replace(r'^\s*$', np.nan, regex=True)
print(len(stars)) 

937000


In [9]:
# Sources with their SpType column including strings of “e”, “s”, “n”, “p”, “f”, “cv”, “i”, “r”, “a”, “D”, “C”, “cont”, “l”,“H”, “h”, “abs”, “+”, “:”, “*”, “?” were removed
# since their spectral type may not be reliable 
stars_d1 = stars[stars['SpType'].str.contains('e|s|n|p|f|cv|i|r|a|D|C|cont|l|H|h|abs|\+|\*|\:|\?', na=False)]
stars_f1 = stars[~stars.set_index(['_RAJ2000','_DEJ2000']).index.isin(stars_d1.set_index(['_RAJ2000','_DEJ2000']).index)]
stars_f1 = stars_f1.reset_index(drop=True)

print(len(stars))
print(len(stars_d1))
print(len(stars_f1))

# any sources with non-empty Remarks column were removed;
stars_d2 = stars_f1[stars_f1['Remarks'].isnull() == False]
stars_f2 = stars_f1[~stars_f1.set_index(['_RAJ2000','_DEJ2000']).index.isin(stars_d2.set_index(['_RAJ2000','_DEJ2000']).index)]
stars_f2 = stars_f2.reset_index(drop=True)

print(len(stars_d2))
print(len(stars_f2))

# Sources with “H97b” in their Name column were removed. They are Orion stars which are likely a mix of faint low-mass stars and YSOs and better to be dropped
stars_d3 = stars_f2[stars_f2['Name'].str.contains('H97b')]
stars_f3 = stars_f2[~stars_f2.set_index(['_RAJ2000','_DEJ2000']).index.isin(stars_d3.set_index(['_RAJ2000','_DEJ2000']).index)]
stars_f3 = stars_f3.reset_index(drop=True)

print(len(stars_d3))
print(len(stars_f3))

937000
238222
622683
69952
516991
692
516182


In [10]:
# Seperate high and low mass stars into their respective classes
star_hm = stars_f3[stars_f3['SpType'].str.startswith(tuple(['O','B','W']), na=False)]
star_lm = stars_f3[stars_f3['SpType'].str.startswith(tuple(['A','F','G','K','M']), na=False)]
star_hm = star_hm.reset_index(drop=True)
star_lm = star_lm.reset_index(drop=True)

star_hm['e_Pos'], star_hm['Class'] = np.nan, 'HM-STAR'
star_lm['e_Pos'], star_lm['Class'] = np.nan, 'LM-STAR'
df_HMSTARs = star_hm.rename(columns={'Name':'name_cat','SpType':'SubClass','Bibcode':'ref'}).drop(columns=['Remarks','Mag'])
df_LMSTARs = star_lm.rename(columns={'Name':'name_cat','SpType':'SubClass','Bibcode':'ref'}).drop(columns=['Remarks','Mag'])
print(len(df_HMSTARs))
print(len(df_LMSTARs))



62124
450224


#### Spectroscopically classified low-mass stars from the APOGEE data in SDSS Data Release 16 were obtained. We filtered out those unreliable sources if they don’t have effective temperature measurements or surface gravity measurements. We also removed those likely binary systems by filtering on the VSCATTER if VSCATTER > 1 km/s and/or VSCATTER > 5*VERR_MED. We also removed sources that are not flagged as a star based on Washington/DDO 51 photometry

In [11]:
APOGEE_all = Vizier(catalog="III/284/allstars",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2020AJ....160..120J
    columns=['*','_RAJ2000', '_DEJ2000','AName','Giant','Star']).query_constraints(Teff='>3000 & <10000',logg='>-1 & <7')[0]

APOGEE = APOGEE_all['_RAJ2000','_DEJ2000','AName','Giant','Star','TClass','Teff','logg','s_HRV','errHRV'].to_pandas()

APOGEE_STAR = APOGEE[(APOGEE.s_HRV <= 1) & (APOGEE.s_HRV <= 5*APOGEE.errHRV) & (APOGEE.Star == 1)].reset_index(drop=True)# & APOGEE.Teff.isnull() & (APOGEE.logg <= 7) & (APOGEE.logg >= -1) ]

APOGEE_STAR['e_Pos'], APOGEE_STAR['Class'], APOGEE_STAR['ref'] = np.nan, 'LM-STAR', '2020AJ....160..120J'
APOGEE_STAR = APOGEE_STAR.rename(columns={'AName':'name_cat','TClass':'SubClass'})
APOGEE_STAR = APOGEE_STAR.replace('none', np.nan, regex=True)
APOGEE_STAR = APOGEE_STAR.drop(columns=['Giant','Star','Teff','logg','s_HRV','errHRV'])
print(len(APOGEE_STAR))

310171


## WRs
### HM-STARs from the VIIth Catalog of Galactic Wolf-Rayet Stars (van der Hucht 2001) and its annex catalog (van der Hucht 2006)

In [12]:
WRs1 = Vizier(catalog="III/215",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2001NewAR..45..135V
    columns=['*', '_RAJ2000', '_DEJ2000','OName']).query_constraints()[0]
WRs1 = WRs1['_RAJ2000','_DEJ2000','Name','OName','Aname'].to_pandas()
WRs1['Class'], WRs1['e_Pos'], WRs1['ref'], WRs1['SubClass'] = 'HM-STAR', np.nan, '2001NewAR..45..135V', np.nan#III/215
WRs1 = WRs1.replace(r'^\s*$', np.nan, regex=True)
WRs1['name_cat'] = WRs1['Name'].combine_first(WRs1['OName'].combine_first(WRs1['Aname']))
df_WRs1 = WRs1.drop(columns=['Name','OName','Aname'])
print(len(df_WRs1))


226


In [13]:
WRs2 = Vizier(catalog="J/A+A/458/453/table1",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2006A%26A...458..453V
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints()[0]
WRs2 = WRs2['_RAJ2000','_DEJ2000','SpType','SpType0','SimbadName','WRori'].to_pandas()
WRs2['Class'], WRs2['e_Pos'], WRs2['ref']= 'HM-STAR', np.nan, '2006A&A...458..453V'#J/A+A/458/453/table1
WRs2 = WRs2.replace(r'^\s*$', np.nan, regex=True)
WRs2['name_cat'] = WRs2['SimbadName'].combine_first(WRs2['WRori'])
WRs2['SubClass'] = WRs2['SpType'].combine_first(WRs2['SpType0'])
df_WRs2 = WRs2.drop(columns=['SpType','SpType0','SimbadName','WRori'])
print(len(df_WRs2))


118


# Quasars & AGNs 
### from Veron Catalog of Quasars & AGN 13th Edition (Veron-Cetty & Veron 2010)

In [14]:
AGNs = Vizier(catalog="VII/258/vv10",row_limit=-1,#https://ui.adsabs.harvard.edu/?#abs/2010A%26A...518A..10V
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints(Cl='|Q|A|B')[0]
AGNs = AGNs['_RAJ2000','_DEJ2000','Name','Cl'].to_pandas()
AGNs['Class'], AGNs['e_Pos'], AGNs['ref']= 'AGN', np.nan, '2010A&A...518A..10V'#VII/258/vv10
df_AGNs = AGNs.rename(columns={'Name':'name_cat','Cl':'SubClass'})

print(len(df_AGNs), Counter(df_AGNs['SubClass']))

168940 Counter({'Q': 133335, 'A': 34231, 'B': 1374})


# HMXBs
### from the Catalog of HMXBs in the Galaxy 4th Edition (Liu et al. 2006)

In [15]:
HMXBs = Vizier(catalog="J/A+A/455/1165/table1",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2006A%26A...455.1165L
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints()[0]
HMXBs = HMXBs['_RAJ2000','_DEJ2000','Name','Type'].to_pandas()
HMXBs['Class'], HMXBs['e_Pos'], HMXBs['ref'] = 'HMXB', np.nan, '2006A&A...455.1165L'#J/A+A/455/1165/table1
df_HMXBs = HMXBs.rename(columns={'Name':'name_cat','Type':'SubClass'})
print(len(df_HMXBs))


114


# LMXBs
### from the Low Mass X-ray Binary Catalog (Liu et al. 2007) and from the Catalog of CVs, LMXBs and related objects (Seventh edition) (Ritter & Kolb 2003)

In [16]:
LMXBs1 = Vizier(catalog="J/A+A/469/807",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2007A%26A...469..807L
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints()[0]
LMXBs1 = LMXBs1['_RAJ2000','_DEJ2000','Name','Type'].to_pandas()
LMXBs1['Class'], LMXBs1['e_Pos'], LMXBs1['ref'] = 'LMXB', np.nan, '2007A&A...469..807L'#J/A+A/469/807
df_LMXBs1 = LMXBs1.rename(columns={'Name':'name_cat','Type':'SubClass'})
print(len(df_LMXBs1))


187


In [17]:
LMXBs2 = Vizier(catalog="B/cb/lmxbdata",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2003A%26A...404..301R
    columns=['*', '_RAJ2000', '_DEJ2000','epos']).query_constraints()[0]

LMXBs2 = LMXBs2['_RAJ2000','_DEJ2000','epos','Name','Type1'].to_pandas()
LMXBs2['Class'], LMXBs2['ref'] = 'LMXB', '2003A&A...404..301R' #B/cb/lmxbdata
df_LMXBs2 = LMXBs2.rename(columns={'Name':'name_cat','Type1':'SubClass','epos':'e_Pos'})
print(len(df_LMXBs2))


108


# CVs
### from the Cataclysmic Variables Catalog 2006 Edition (Downes et al. 2001) and the Catalog of CVs, LMXBs and related objects (Seventh edition) (Ritter & Kolb 2003)

In [18]:
CVs1 = Vizier(catalog="V/123A",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2001PASP..113..764D
    columns=['*', '_RAJ2000', '_DEJ2000']).query_constraints()[0]
CVs1 = CVs1['_RAJ2000','_DEJ2000','Names','VarType'].to_pandas()
CVs1['Class'], CVs1['e_Pos'], CVs1['ref'] = 'CV', np.nan, '2005JAD....11....2D'#'2001PASP..113..764D'#V/123A
CVs1 = CVs1[CVs1['VarType']!='non-CV'].reset_index(drop=True)
df_CVs1 = CVs1.rename(columns={'Names':'name_cat','VarType':'SubClass'})
print(len(df_CVs1))


1618


In [19]:
CVs2 = Vizier(catalog="B/cb/cbdata",row_limit=-1, #https://ui.adsabs.harvard.edu/?#abs/2003A%26A...404..301R
    columns=['*', '_RAJ2000', '_DEJ2000','epos']).query_constraints()[0]
CVs2 = CVs2['_RAJ2000','_DEJ2000','epos','Name','Type1'].to_pandas()
CVs2['Class'], CVs2['ref'] = 'CV', '2003A&A...404..301R'#B/cb/cbdata
df_CVs2 = CVs2.rename(columns={'Name':'name_cat','Type1':'SubClass','epos':'e_Pos'})
print(len(df_CVs2))


1429


# NS & NS_BIN
### from the ATNF Pulsar Catalog (Manchester et al. 2005)

In [20]:
import urllib3
#https://ui.adsabs.harvard.edu/abs/2005AJ....129.1993M/abstract
http = urllib3.PoolManager()
r = http.request('GET', 'https://www.atnf.csiro.au/research/pulsar/psrcat/proc_form.php?version=1.65&Name=Name&RaJ=RaJ&DecJ=DecJ&Binary=Binary&Type=Type&startUserDefined=true&c1_val=&c2_val=&c3_val=&c4_val=&sort_attr=jname&sort_order=asc&condition=&pulsar_names=&ephemeris=short&coords_unit=raj%2Fdecj&radius=&coords_1=&coords_2=&style=Long+csv+with+errors&no_value=*&fsize=3&x_axis=&x_scale=linear&y_axis=&y_scale=linear&state=query&table_bottom.x=35&table_bottom.y=15') # it's a file like object and works just like a file
r.status

ATNF = r.data.decode('utf-8').partition('\n<pre>\n')[2].partition('\n</pre>\n')[0].replace('*',' ').split('\n')
NSs = pd.DataFrame(columns=['src', 'NAME','Name_ref','RAJ','e_RA','RAJ_ref','DECJ','e_DEC','DECJ_ref','Binary','Binary_ref','PSR_type','Type_ref'], 
                data=[row.split(';') for row in ATNF[2:]])

NSs['_RAJ2000'] = NSs.apply(lambda row: atnf_pos(row.RAJ, row.e_RA, 'hms', 'pos'), axis=1)
NSs['_e_RAJ2000'] = NSs.apply(lambda row: atnf_pos(row.RAJ, row.e_RA, 'hms', 'err'), axis=1)
NSs['_DEJ2000'] = NSs.apply(lambda row: atnf_pos(row.DECJ, row.e_DEC, 'dms', 'pos'), axis=1)
NSs['_e_DEJ2000'] = NSs.apply(lambda row: atnf_pos(row.DECJ, row.e_DEC, 'dms', 'err'), axis=1)
NSs['e_Pos'] = NSs.apply(lambda row: max(row._e_RAJ2000 , row._e_DEJ2000), axis=1)


# correcting the inaccurate coordinates of two NSs
NSs.loc[NSs.NAME=='J1819-1458', '_RAJ2000'] = 274.8924
NSs.loc[NSs.NAME=='J1819-1458', '_DEJ2000'] = -14.9676579999999
NSs.loc[NSs.NAME=='J1741-2054', '_RAJ2000'] = 265.48868
NSs.loc[NSs.NAME=='J1741-2054', '_DEJ2000'] = -20.903278

# non-empty Binary column are binary non-accreting NSs (NS_BIN class) and share similar properties with LMXBs.
NSs['ref']= '2005AJ....129.1993M'#B/psr/psr
NSs['Class'] = 'NS_BIN'
print(len(NSs),Counter(NSs['Binary']))
NSs.loc[NSs['Binary']==' ', 'Class'] = 'NS'

# adding a few new NSs and NS_BINs
new_NS_BINs = pd.read_csv(f'{data_dir}/new_NS_BIN.csv')
print(NSs.loc[NSs.NAME.isin(new_NS_BINs.name_cat.values), ['NAME','Binary','Class']])

df_NSs = NSs[['NAME','_RAJ2000','_DEJ2000','e_Pos','Class','PSR_type','ref']].rename(columns={'NAME':'name_cat','PSR_type':'SubClass'})
print(len(df_NSs))

print(len(df_NSs),Counter(df_NSs['Class']))


3177 Counter({' ': 2844, 'ELL1': 143, 'BT': 107, 'DD': 39, 'DDH': 14, 'ELL1H': 10, 'BTX': 7, 'DDGR': 4, 'T2': 3, 'MSS': 3, 'DDS': 1, 'BT2P': 1, 'DDK': 1})
             NAME Binary   Class
6      J0023+0923   ELL1  NS_BIN
60     J0101-6422     BT  NS_BIN
313   J0737-3039B     DD  NS_BIN
560    J1124-3653             NS
644    J1231-1411     BT  NS_BIN
714    J1311-3430   ELL1  NS_BIN
921    J1514-4946   ELL1  NS_BIN
984      B1534+12     DD  NS_BIN
1107   J1614-2230  ELL1H  NS_BIN
1172   J1628-3205     BT  NS_BIN
1294   J1653-0158   ELL1  NS_BIN
1494   J1731-1847    BTX  NS_BIN
1836   J1810+1744     BT  NS_BIN
1881   J1816+4510   ELL1  NS_BIN
2545   J1909-3744   ELL1  NS_BIN
2909     B1957+20     BT  NS_BIN
2953   J2017+0603  ELL1H  NS_BIN
3002   J2043+1711   ELL1  NS_BIN
3013   J2047+1053     BT  NS_BIN
3018   J2051-0827   ELL1  NS_BIN
3100   J2214+3000   ELL1  NS_BIN
3102   J2215+5135   ELL1  NS_BIN
3106   J2222-0137     DD  NS_BIN
3125   J2241-5236     BT  NS_BIN
3133   J2256-1024   

# HMXBs, LMXBs, and CVs from INTEGRAL General Reference Catalog (IGRS) and HMXBs from Be Star catalog

In [21]:
df_HMXB_Be = pd.read_csv(f'{data_dir}/raretype_BeStar_IGRS.csv')
print(Counter(df_HMXB_Be['Class']))

Counter({'HMXB': 55, 'LMXB': 8, 'CV': 5})


# Combining sources together

In [22]:
df_TD = pd.concat([df_AGNs, df_YSOs, df_LMSTARs, APOGEE_STAR, df_HMSTARs, df_WRs1, df_WRs2, df_NSs, df_HMXBs, df_LMXBs1, df_LMXBs2, df_CVs1, df_CVs2, df_HMXB_Be], ignore_index=True, sort=False)

In [23]:
df_TD.head(5)

,_RAJ2000,_DEJ2000,name_cat,SubClass,Class,e_Pos,ref
0,0.005417,-2.033333,FIRST J00000-0202,Q,AGN,NaN,2010A&A...518A..10V
1,0.005833,-30.607500,2QZ J000001-3036,Q,AGN,NaN,2010A&A...518A..10V
2,0.007083,-31.373889,2QZ J000001-3122,Q,AGN,NaN,2010A&A...518A..10V
3,0.011250,-25.193611,XMM J00000-2511,Q,AGN,NaN,2010A&A...518A..10V
4,0.011667,-35.059167,MS 23574-3520,Q,AGN,NaN,2010A&A...518A..10V


In [24]:
print(len(df_TD), sorted(Counter(df_TD['Class']).items()))


1003439 [('AGN', 168940), ('CV', 3052), ('HM-STAR', 62468), ('HMXB', 169), ('LM-STAR', 760395), ('LMXB', 303), ('NS', 2844), ('NS_BIN', 333), ('YSO', 4935)]


In [25]:
# matching with CSCv2 

TD_CSC = XMatch.query(cat1= Table.from_pandas(df_TD), #open('/Users/yanghui/Desktop/Research/2019/MUWCLASS_Project/ML/DATA/TD/versions/CSC_TD_v5_09062021.csv'),
                      cat2='vizier:IX/57/csc2master',
                      max_distance=3*u.arcsec, colRA1='_RAJ2000',colDec1='_DEJ2000')

TD_CSC = TD_CSC.to_pandas()
print(len(TD_CSC), sorted(Counter(TD_CSC['Class']).items()))


11331 [('AGN', 6174), ('CV', 342), ('HM-STAR', 895), ('HMXB', 77), ('LM-STAR', 1862), ('LMXB', 166), ('NS', 222), ('NS_BIN', 148), ('YSO', 1445)]


In [26]:
TD_CSC.head(5)

TD_CSC.to_csv(f'{data_dir}/{field_name}_Xmatch_all.csv',index=False)



In [57]:
TD_CSC = pd.read_csv(f'{data_dir}/{field_name}_Xmatch_all.csv')

# drop duplicated sources
TD_CSC = TD_CSC.sort_values(by=['angDist']) 
TD_CSC = TD_CSC.drop_duplicates(subset=['_RAJ2000_1', '_DEJ2000_1', 'name_cat', 'SubClass', 'Class','e_Pos', 'ref']).reset_index(drop=True)

print(len(TD_CSC), sorted(Counter(TD_CSC['Class']).items()))


10723 [('AGN', 5978), ('CV', 275), ('HM-STAR', 767), ('HMXB', 68), ('LM-STAR', 1834), ('LMXB', 139), ('NS', 165), ('NS_BIN', 90), ('YSO', 1407)]


In [58]:
# calculate the combined positional uncertainties (PUs) from X-ray positions and class-specific catalog coordinates 
TD_CSC['PU'] = np.sqrt(TD_CSC.e_Pos.fillna(0)*2**2+TD_CSC.r0.fillna(0)**2)

TD_CSC['name'] = TD_CSC.apply(lambda row: '2CXO '+str(row['2CXO']),axis=1)

# Sources from populous classes (AGNs, HM-STARs, LM-STARs and YSOs) are omitted if their class-specific catalog 
# and X-ray combined 2-σ PUs are > 1" or 
# if the separations of the class-specific catalog and the CSCv2 coordinates exceed the 2-σ PUs.

idx = np.where( ((TD_CSC['angDist']>TD_CSC['PU']) | (TD_CSC['PU'] >1.) )& ((TD_CSC['Class']=='AGN') | (TD_CSC['Class']=='YSO') | (TD_CSC['Class']=='HM-STAR') | (TD_CSC['Class']=='LM-STAR') ))[0]
print('Remove', len(idx), sorted(Counter(TD_CSC.loc[idx, 'Class']).items()))
TD_CSC = TD_CSC.drop(TD_CSC.index[idx])
TD_CSC = TD_CSC.reset_index(drop=True)
print(len(TD_CSC), sorted(Counter(TD_CSC['Class']).items()))

TD = TD_CSC.rename(columns={'_RAJ2000_1':'ra_cat','_DEJ2000_1':'dec_cat','angDist':'sep','RAICRS':'ra','DEICRS':'dec'})[['name_cat','ra_cat','dec_cat','e_Pos','Class','SubClass','ref','sep','name','ra','dec','r0','PU','S/N']].sort_values(by=['Class','ra']).reset_index(drop=True)


TD['remove_code'] = 0



Remove 6440 [('AGN', 4485), ('HM-STAR', 364), ('LM-STAR', 1108), ('YSO', 483)]
4283 [('AGN', 1493), ('CV', 275), ('HM-STAR', 403), ('HMXB', 68), ('LM-STAR', 726), ('LMXB', 139), ('NS', 165), ('NS_BIN', 90), ('YSO', 924)]


In [59]:
# cross-matching to SIMBAD 

TD_simbad = XMatch.query(cat1=Table.from_pandas(TD),
                         cat2='vizier:SIMBAD',max_distance=3 * u.arcsec, colRA1='ra',colDec1='dec')
TD_simbad = TD_simbad.to_pandas().rename(columns={'ra_1':'ra', 'dec_1':'dec','ra_2':'ra_simbad', 'dec_2':'dec_simbad', 'angDist':'_r_simbad'}).sort_values(by=['_r_simbad']) 
TD_simbad = TD_simbad.drop_duplicates(subset=['name_cat','ra_cat','dec_cat','Class','name'], keep='first').reset_index(drop=True)
print(len(TD_simbad))
TD= pd.merge(TD, TD_simbad, how='outer', on = ['name_cat','ra_cat','dec_cat','e_Pos','Class','SubClass','ref', 'sep', 'name', 'ra', 'dec', 'r0','PU','S/N','remove_code'])

TD.loc[TD.name_cat.isnull(), 'name_cat'] = TD.loc[TD.name_cat.isnull(), 'main_id']
TD.loc[TD.name_cat.isnull(), 'name_cat'] = TD.loc[TD.name_cat.isnull(), 'name']
print(TD.columns[:50])


4105
Index(['name_cat', 'ra_cat', 'dec_cat', 'e_Pos', 'Class', 'SubClass', 'ref',
       'sep', 'name', 'ra', 'dec', 'r0', 'PU', 'S/N', 'remove_code',
       '_r_simbad', 'main_id', 'ra_simbad', 'dec_simbad', 'coo_err_maj',
       'coo_err_min', 'coo_err_angle', 'nbref', 'ra_sexa', 'dec_sexa',
       'coo_qual', 'coo_bibcode', 'main_type', 'other_types', 'radvel',
       'radvel_err', 'redshift', 'redshift_err', 'sp_type', 'morph_type',
       'plx', 'plx_err', 'pmra', 'pmdec', 'pm_err_maj', 'pm_err_min',
       'pm_err_pa', 'size_maj', 'size_min', 'size_angle', 'B', 'V', 'R', 'J',
       'H'],
      dtype='object')


In [60]:
## Re-assign those STARs which are actually YSOs 

## Some (LM- or HM-) STARs are YSOs and misclassfied as STARs, including those with their main type column 
## containing “Orion V” or “TTau” or “YSO”, or if their names containing “MBO 62”, 
## or if they are included in the catalog from CL Slesnick+2004 (which includes a list of candidate brown dwarf members of the Orion Nebula cluster).

s = np.where(((TD['main_type'].str.contains('Orion_V*|TTau*|YSO')) & (TD['Class']!='YSO')) | (TD['name_cat'].isin(['MBO 62'])) | (TD['ref']=='2004ApJ...610.1045S') )[0] 
print("Converting", sorted(Counter(TD['Class'][s]).items()), "sources to YSOs.")
TD.loc[s, 'Class'] = 'YSO'
print(len(TD), sorted(Counter(TD['Class']).items()))


Converting [('HM-STAR', 91), ('LM-STAR', 291)] sources to YSOs.
4283 [('AGN', 1493), ('CV', 275), ('HM-STAR', 312), ('HMXB', 68), ('LM-STAR', 435), ('LMXB', 139), ('NS', 165), ('NS_BIN', 90), ('YSO', 1306)]


### a few sources are misclassified after manual investigation of rare type sources

In [61]:
print(TD.loc[TD.name_cat.isin(['B1259-63','J1124-3653','J2032+4127','XTE J1858+034','J0737-3039B','B1534+12']), ['name_cat','Class']])

TD.loc[TD.name_cat=='B1259-63', 'Class'] = 'HMXB'
TD.loc[TD.name_cat=='J1124-3653', 'Class'] = 'NS_BIN'
TD.loc[TD.name_cat=='J2032+4127', 'Class'] = 'HMXB'
TD.loc[TD.name_cat=='XTE J1858+034', 'Class'] = 'LMXB'

double_NSs = ['J0737-3039B', 'B1534+12']
TD.loc[TD.name_cat.isin(double_NSs), 'Class'] = 'NS'

print(TD.loc[TD.name_cat.isin(['B1259-63','J1124-3653','J2032+4127','XTE J1858+034','J0737-3039B','B1534+12']), ['name_cat','Class']])

           name_cat   Class
2225  XTE J1858+034    HMXB
3152     J1124-3653      NS
3287    J0737-3039B  NS_BIN
3292       B1259-63  NS_BIN
3297       B1534+12  NS_BIN
3349     J2032+4127  NS_BIN
           name_cat   Class
2225  XTE J1858+034    LMXB
3152     J1124-3653  NS_BIN
3287    J0737-3039B      NS
3292       B1259-63    HMXB
3297       B1534+12      NS
3349     J2032+4127    HMXB


### removing duplicated CSCv2 sources (remove code=2)

In [62]:

# X-ray sources are removed if they are matched to multiple literature verified sources (LVSs):
# 1. if different LVSs with the same classification are matched the same CSCv2 source, the nearest LVS is remained. 
# 2. if CSCv2 sources are matched to LVSs with different classifications, the X-ray sources will be removed. 
print("remove code = 2 duplicate CSC sources")

TD_dup = TD[TD.duplicated(subset=['name'], keep=False)].sort_values(by=['name'])


s_all = []
for name in TD_dup.name.unique():
    df_s = TD[TD['name']==name]
    df_s = df_s.reset_index(drop=True)
    if len(df_s['Class'].unique()) >1:
        s = np.where(TD['name']==name)[0]
        s_all = np.append(s_all, s)
    else:
        
        sep_min = min(df_s['sep'])
        s_src = np.where(TD['name']==name)[0]
        s_min = np.where( (TD['name']==name) & (TD['sep']==sep_min))[0]
        
        if len(s_min)>1:
            s_all = np.append(s_all, list(set(s_src) - set([s_min[0]])))
        else:
            s_all = np.append(s_all, list(set(s_src) - set(s_min)))
        

TD.loc[s_all, 'remove_code'] = TD.loc[s_all, 'remove_code']+2
print('Remove', len(s_all), sorted(Counter(TD.loc[s_all, 'Class']).items()))
print('Left', len(TD[TD['remove_code']==0]), sorted(Counter(TD[TD['remove_code']==0]['Class']).items()))



remove code = 2 duplicate CSC sources
Remove 609 [('CV', 78), ('HM-STAR', 156), ('HMXB', 9), ('LM-STAR', 113), ('LMXB', 44), ('NS', 32), ('NS_BIN', 35), ('YSO', 142)]
Left 3674 [('AGN', 1493), ('CV', 197), ('HM-STAR', 156), ('HMXB', 60), ('LM-STAR', 322), ('LMXB', 96), ('NS', 134), ('NS_BIN', 52), ('YSO', 1164)]


In [63]:
print("remove_code = 4: sources in crowded/complex environments")

# removing sources from the globular clusters from Kharchenko, N. V et al. 2013
viz = Vizier(row_limit=-1,  timeout=5000, columns=["**", "+_r"], catalog="J/A+A/558/A53/catalog",column_filters={"Type":"g"},)

radec = [[TD.loc[i, 'ra'], TD.loc[i, 'dec']] for i in range(len(TD))]
rd = Table(Angle(radec, 'deg'), names=('_RAJ2000', '_DEJ2000'))

start = time.time()
query_res = viz.query_region(rd, radius=1.*u.deg)[0]
df_gc = query_res.to_pandas()
df_gc = df_gc[df_gc._r < df_gc.r2].reset_index(drop=True)
df_gc['_q_index'] = df_gc['_q']-1
end = time.time() 
print(end - start)

TD['remove_4'] = 0
TD.loc[TD.index.isin(df_gc._q_index), 'remove_4'] = TD.loc[TD.index.isin(df_gc._q_index), 'remove_4'] +1


remove_code = 4: sources in crowded/complex environments
0.42588257789611816


In [64]:
# calculate Galactic coordinates

TD['Gal_Long'] = TD.apply(lambda row: SkyCoord(ra=row.ra*u.degree, dec=row.dec*u.degree, frame='icrs').galactic.l.degree, axis=1)

TD['Gal_Lat'] =  TD.apply(lambda row: SkyCoord(ra=row.ra*u.degree, dec=row.dec*u.degree, frame='icrs').galactic.b.degree, axis=1)


In [65]:
# removing sources from a list of crowded environments  

#          LMC,      SMC,  Westerlund 1, M31, NGC 300, NGC 3379, Liller 1,NGC 6791, M 27/NGC 6853, Circinus Galaxy, NGC 2264, IC 348, NGC 1333      
re_ras =[80.89375, 13.1867, 251.76667,10.68458,13.7229,161.95666,263.3520,290.22083,299.901417, 213.29125,        100.25,  56.13833, 52.2971]
re_decs=[-69.75611,-72.8286,-45.85136,41.26916,-37.6844,12.58163,-33.3889,37.771667,22.721136, -65.339167,        9.8833,  32.16333, 31.31]
re_rs  =[10.75,    5.33,    3./60,    1.,      0.17,     0.04,   0.003,   16./60,  8./60,     6.9/60,             45./60,  42./60,   6./60]

for ra, dec, r in zip(re_ras, re_decs, re_rs):
    #print(ra, dec, r)
    TD['remove_4'] = TD.apply(lambda row: row.remove_4 +1 if SkyCoord(row.ra*u.deg, row.dec*u.deg, frame='icrs').separation(SkyCoord(ra*u.deg, dec*u.deg, frame='icrs')).deg < r else row.remove_4,axis=1)

# removing sources from the crowded Galactic Center
idx = np.where(((TD.Gal_Long > 350.) | (TD.Gal_Long < 10.)) & (TD.Gal_Lat>-5.) & (TD.Gal_Lat<5.))[0]# & (TD.Class!='LM-STAR'))[0]
TD.loc[idx, 'remove_4'] = TD.loc[idx, 'remove_4']+1

idx = np.where((TD['name'].str.strip().str[-1].str.isalpha()) & (~TD['name'].isin(['2CXO J043715.9-471509X'])))[0]
TD.loc[idx, 'remove_4'] = TD.loc[idx, 'remove_4'] + 1

# drop sources (mostly NSs) in complex exvironment, in bright PWNe 
SN_delete = ['2CXO J151355.6-590809','2CXO J174715.8-295801','2CXO J183333.5-103407','2CXO J053747.4-691019','2CXO J083520.6-451034','2CXO J195258.2+325240','2CXO J054010.8-691954','2CXO J090835.4-491305','2CXO J180150.6-085733','2CXO J184343.3-040805','2CXO J102347.6+003840','2CXO J174615.5-321400','2CXO J163905.4-464212']

# LMXBs and HMXBs that appeared in the previous TD and most of them are in the crowded environment but since they appeared in the 
# previous TD so they have gone through manual investigation so should be fine to add
LMXBs_HMXBs_save = ['2CXO J174819.2-360716', '2CXO J180632.1-221417', '2CXO J181044.4-260901', '2CXO J174502.3-285449', 
              '2CXO J174702.5-285259', '2CXO J173413.4-260518', '2CXO J174931.7-280805', '2CXO J173953.9-282946',
              '2CXO J174433.0-284426', '2CXO J174354.8-294443',  '2CXO J171419.7-340246','2CXO J174451.1-292116', 
              '2CXO J181921.6-252425', '2CXO J174621.1-284343', # LMXB so far
              '2CXO J173527.5-325554', '2CXO J175834.5-212321', '2CXO J174445.7-271344'] # HMXBs


s = np.where(((TD['remove_4']>0) | (TD.name.isin(SN_delete))) & (~TD.name.isin(LMXBs_HMXBs_save)) )[0]
TD.loc[s, 'remove_code'] = TD.loc[s, 'remove_code'] + 4

print('Remove', len(s), sorted(Counter(TD.loc[s, 'Class']).items()))
print('Left', len(TD[TD['remove_code']==0]), sorted(Counter(TD[TD['remove_code']==0]['Class']).items()))



Remove 659 [('AGN', 9), ('CV', 181), ('HM-STAR', 67), ('HMXB', 15), ('LM-STAR', 79), ('LMXB', 55), ('NS', 64), ('NS_BIN', 59), ('YSO', 130)]
Left 3216 [('AGN', 1484), ('CV', 58), ('HM-STAR', 121), ('HMXB', 45), ('LM-STAR', 265), ('LMXB', 58), ('NS', 101), ('NS_BIN', 25), ('YSO', 1059)]


In [66]:
print("remove_code = 8: delete sources with ambiguous classifications")

src_delete = ['[DWS84] 20', 'HD 38563 C', '[RHI84] 10- 632', '[HC2000] 114','[CHG2008] C2-18','Cl* NGC 2244 John 14','V* V2361 Ori','V* V1129 Cen','HD 35914','M 51 X-7','1E 161348-5055.1','XTE J1829-098']

s = np.where( (TD['name_cat'].isin(src_delete)) | (TD['main_type'] == 'Candidate_YSO'))[0]
TD.loc[s, 'remove_code'] = TD.loc[s, 'remove_code'] + 8

print('Remove', len(s), sorted(Counter(TD.loc[s, 'Class']).items()))
print('Left', len(TD[TD['remove_code']==0]), sorted(Counter(TD[TD['remove_code']==0]['Class']).items()))


remove_code = 8: delete sources with ambiguous classifications
Remove 13 [('CV', 1), ('HM-STAR', 3), ('HMXB', 1), ('LM-STAR', 3), ('LMXB', 2), ('YSO', 3)]
Left 3207 [('AGN', 1484), ('CV', 57), ('HM-STAR', 120), ('HMXB', 44), ('LM-STAR', 263), ('LMXB', 56), ('NS', 101), ('NS_BIN', 25), ('YSO', 1057)]


In [67]:
TD.to_csv(f'{data_dir}/{field_name}.csv',index=False)

In [4]:
TD = pd.read_csv(f'{data_dir}/{field_name}.csv')
TD = TD[TD['remove_code']==0].reset_index(drop=True)

In [5]:

print(len(TD),Counter(TD.Class))



3207 Counter({'AGN': 1484, 'YSO': 1057, 'LM-STAR': 263, 'HM-STAR': 120, 'NS': 101, 'CV': 57, 'LMXB': 56, 'HMXB': 44, 'NS_BIN': 25})


In [6]:
TD.columns

Index(['name_cat', 'ra_cat', 'dec_cat', 'e_Pos', 'Class', 'SubClass', 'ref',
       'sep', 'name', 'ra', 'dec', 'r0', 'PU', 'S/N', 'remove_code',
       '_r_simbad', 'main_id', 'ra_simbad', 'dec_simbad', 'coo_err_maj',
       'coo_err_min', 'coo_err_angle', 'nbref', 'ra_sexa', 'dec_sexa',
       'coo_qual', 'coo_bibcode', 'main_type', 'other_types', 'radvel',
       'radvel_err', 'redshift', 'redshift_err', 'sp_type', 'morph_type',
       'plx', 'plx_err', 'pmra', 'pmdec', 'pm_err_maj', 'pm_err_min',
       'pm_err_pa', 'size_maj', 'size_min', 'size_angle', 'B', 'V', 'R', 'J',
       'H', 'K', 'u', 'g', 'r', 'i', 'z', 'remove_4', 'Gal_Long', 'Gal_Lat'],
      dtype='object')

In [7]:
TD[TD['S/N']<3]

,name_cat,ra_cat,dec_cat,e_Pos,Class,SubClass,ref,sep,name,ra,...,H,K,u,g,r,i,z,remove_4,Gal_Long,Gal_Lat
0,SDSS J00001+1356,0.039167,13.938333,NaN,AGN,Q,2010A&A...518A..10V,0.604358,2CXO J000009.3+135618,0.039115,...,18.206,18.183,19.238,18.895,18.439,18.314,18.101,0,104.492215,-47.088916
16,UM 245,6.890833,-1.581111,NaN,AGN,Q,2010A&A...518A..10V,0.585460,2CXO J002733.7-013452,6.890816,...,15.736,14.942,NaN,NaN,NaN,NaN,NaN,0,109.305155,-63.819570
24,XMM J00335-4320,8.383750,-43.335556,NaN,AGN,A,2010A&A...518A..10V,0.587811,2CXO J003332.0-432007,8.383530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,314.386874,-73.392273
25,XMM J00335-4322,8.392917,-43.372500,NaN,AGN,A,2010A&A...518A..10V,0.380255,2CXO J003334.3-432220,8.392976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,314.332323,-73.358159
27,XMM J00338-4321,8.473333,-43.357778,NaN,AGN,A,2010A&A...518A..10V,0.525442,2CXO J003353.5-432127,8.473139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,314.146917,-73.386381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,2MASS J10453801-6012319,161.408375,-60.208917,NaN,YSO,II,2011ApJS..194...14P,0.366272,2CXO J104538.0-601232,161.408496,...,14.295,13.514,NaN,NaN,NaN,NaN,NaN,0,287.904649,-1.060318
3150,2MASS J10455662-5958388,161.485917,-59.977444,NaN,YSO,II,2011ApJS..194...14P,0.280492,2CXO J104556.5-595838,161.485764,...,11.860,11.462,NaN,NaN,NaN,NaN,NaN,0,287.831353,-0.837331
3155,2MASS J10460614-5958094,161.525458,-59.969278,NaN,YSO,II,2011ApJS..194...14P,0.299267,2CXO J104606.1-595809,161.525555,...,11.786,11.315,NaN,NaN,NaN,NaN,NaN,0,287.845243,-0.820917
3168,2MASS J10483627-6014086,162.151125,-60.235722,NaN,YSO,0/I,2011ApJS..194...14P,0.459962,2CXO J104836.2-601408,162.150868,...,15.196,14.411,NaN,NaN,NaN,NaN,NaN,0,288.244440,-0.914645


In [19]:
# extract per-observation data from CSCv2 

df_pers = create_perobs_data(TD, query_dir, data_dir, name_type='CSCview', name_col='name', ra_col='ra',dec_col='dec',coord_format='deg')
df_pers.to_csv(f'{data_dir}/{field_name}_per.csv', index=False)

In [8]:
# Calculate the weighted average fluxes and variability parameters for X-ray properties
# Will generate warnings but they can be ignored.

df_pers = pd.read_csv(f'{data_dir}/{field_name}_per.csv', low_memory=False)

df_pers['name'] = df_pers['name'].str.lstrip()
df_pers['per_remove_code'] = 0

df_ave, df_obs = cal_ave(df_pers, data_dir, dtype='TD',Chandratype='CSC',verb=verb)


df_ave = pd.merge(df_ave, TD.iloc[:, np.r_[:9, 15:17, 27]], how='inner', on='name')
print(Counter(df_ave.Class))
df_ave.to_csv(f'{data_dir}/{field_name}_ave.csv', index=False)



There are 16046 per-obs data.
Run add_newdata......
Before adding new data:
Run stats......
 H   M   S    #    %  
--- --- --- ----- ----
  Y   Y   Y 13128 81.8
  Y   Y   N   867  5.4
  N   Y   Y   304  1.8
  Y   N   Y   248  1.5
  N   Y   N    68  0.4
  Y   N   N   377  2.3
  N   N   Y   111  0.6
  N   N   N   943  5.8
 ~Y   Y   Y  2918 18.1
-----------------
total:      16046
Only  13128  detections have valid fluxes at all bands.
After adding new  2289 s band data:
Run stats......
 H   M   S    #    %  
--- --- --- ----- ----
  Y   Y   Y 13635 84.9
  Y   Y   N   360  2.2
  N   Y   Y   336  2.0
  Y   N   Y   468  2.9
  N   Y   N    36  0.2
  Y   N   N   157  0.9
  N   N   Y   229  1.4
  N   N   N   825  5.1
 ~Y   Y   Y  2411 15.0
-----------------
total:      16046
Only  13635  detections have valid fluxes at all bands.
After adding new  1420 m band data:
Run stats......
 H   M   S    #    %  
--- --- --- ----- ----
  Y   Y   Y 13938 86.8
  Y   Y   N   361  2.2
  N   Y   Y   475  2.9

In [9]:
print(len(df_ave),Counter(df_ave['Class']))

3012 Counter({'AGN': 1390, 'YSO': 1039, 'LM-STAR': 242, 'HM-STAR': 117, 'NS': 87, 'CV': 44, 'LMXB': 42, 'HMXB': 26, 'NS_BIN': 25})


In [10]:
print(df_ave['flux_flag'].value_counts())
df_ave_pre = pd.read_csv(f'{data_dir}/CSCv2_TD_ave.csv')
print(df_ave_pre['flux_flag'].value_counts())

0    3012
Name: flux_flag, dtype: int64
0    2830
1      76
4      58
3      21
2      16
6       8
7       2
5       1
Name: flux_flag, dtype: int64


### Cross-matching to multi-wavelength catalogs

In [49]:
#THIS PART TAKES several hours to run for the first time as it is performing all cross-matching. After that it saves to cache and only takes ~200 seconds to run.

df_ave = pd.read_csv(f'{data_dir}/{field_name}_ave.csv')

# cross-match with MW catalogs
start = time.time()
add_MW(df_ave, data_dir, field_name, Chandratype='CSC')
end = time.time() 
print(end - start)



I/350/gaiaedr3
cross-matching to gaia
cross-matching to gaiadist
II/246/out
cross-matching to 2mass
II/365/catwise
cross-matching to catwise
II/363/unwise
cross-matching to unwise
II/328/allwise
cross-matching to allwise
230.51056265830994


In [12]:
### cleaning the counterpart based on the combined PUs from X-ray sources and multiwavelength counterparts 

In [50]:
df_MW = pd.read_csv(f'{data_dir}/{field_name}_MW.csv')

# correcting a source's PU so that the correct counterpart can be matched
df_MW.loc[df_MW.name == '2CXO J182615.0-145055', 'err_ellipse_r0'] = 1.2 

df_MW_cf = confusion_clean(df_MW,X_PU='err_ellipse_r0',Chandratype='CSC')
df_MW_cf.to_csv(f'{data_dir}/{field_name}_MW_clean.csv',index=False)

df_MW_cf = pd.read_csv(f'{data_dir}/{field_name}_MW_clean.csv')
df_ave = TD_clean(df_MW_cf, remove_codes = [1, 32]) 

# removing a few unreliable sources
s_remove = np.where(df_ave.name_cat.isin(['Mon R2- 5b','Cl* NGC 2264    SBL    1025B','** RAT   17A','NGC 6611 374','NGC 6611 245','NGC 6611 213','2MASS J13121845-6237309','V* V700 Per','ESO-HA 1171', 'VSS II- 7']))[0]

df_ave.loc[s_remove, 'remove_code'] = df_ave.loc[s_remove, 'remove_code']+128

df_ave.to_csv(f'{data_dir}/{field_name}_MW_before_remove.csv', index=False)

df_remove = df_ave[df_ave['remove_code']==0].reset_index(drop=True)
print('Final breakdown', len(df_remove), Counter(df_remove['Class']))
df_remove.to_csv(f'{data_dir}/{field_name}_MW_remove.csv', index=False)




2275 counterparts matched for gaia
1854 counterparts matched for 2mass
1909 counterparts matched for catwise
2117 counterparts matched for unwise
1810 counterparts matched for allwise
[]
[('AGN', 1390), ('CV', 44), ('HM-STAR', 117), ('HMXB', 26), ('LM-STAR', 242), ('LMXB', 42), ('NS', 87), ('NS_BIN', 24), ('YSO', 1039)]
Final breakdown 3003 Counter({'AGN': 1390, 'YSO': 1039, 'LM-STAR': 235, 'HM-STAR': 116, 'NS': 87, 'CV': 44, 'LMXB': 42, 'HMXB': 26, 'NS_BIN': 24})


In [51]:
print(len(df_MW_cf),Counter(df_MW_cf['Class']))

print(Counter(df_ave['remove_code']))

3012 Counter({'AGN': 1390, 'YSO': 1039, 'LM-STAR': 242, 'HM-STAR': 117, 'NS': 87, 'CV': 44, 'LMXB': 42, 'HMXB': 26, 'NS_BIN': 25})
Counter({0: 3003, 128: 8, 1: 1})


In [53]:
df = pd.read_csv(f'{data_dir}/{field_name}_MW_remove.csv')
df_final = prepare_cols(df, cp_thres=0, vphas=False,gaiadata=False,cp_conf_flag=False, TD=True, NS_MWdrop=False, STAR_classremove=['HM-STAR','LM-STAR','YSO'])
df_pre = pd.read_csv(f'{data_dir}/CSCv2_TD_MW_remove.csv')
df_final_pre = prepare_cols(df_pre, cp_thres=0, vphas=False,gaiadata=False,cp_conf_flag=False, TD=True, NS_MWdrop=False, STAR_classremove=['HM-STAR','LM-STAR','YSO'])


Remove 25 [('LM-STAR', 4), ('YSO', 21)]
Final breakdown 2978 [('AGN', 1390), ('CV', 44), ('HM-STAR', 116), ('HMXB', 26), ('LM-STAR', 231), ('LMXB', 42), ('NS', 87), ('NS_BIN', 24), ('YSO', 1018)]
Remove 25 [('LM-STAR', 4), ('YSO', 21)]
Final breakdown 2977 [('AGN', 1390), ('CV', 44), ('HM-STAR', 116), ('HMXB', 26), ('LM-STAR', 232), ('LMXB', 41), ('NS', 87), ('NS_BIN', 24), ('YSO', 1017)]


In [60]:
df_com = pd.merge(df[['name','ra','dec','dof','Class','flux_aper90_ave_s', 'e_flux_aper90_ave_s', 'flux_aper90_ave_m','e_flux_aper90_ave_m', 'flux_aper90_ave_h', 'e_flux_aper90_ave_h','flux_aper90_ave_b','e_flux_aper90_ave_b']],df_pre[['name','Class','dof','flux_aper90_ave_s', 'e_flux_aper90_ave_s', 'flux_aper90_ave_m','e_flux_aper90_ave_m', 'flux_aper90_ave_h', 'e_flux_aper90_ave_h','flux_aper90_ave_b','e_flux_aper90_ave_b']],on='name',how='outer')
print(len(df_com))

for col in ['flux_aper90_ave_s',  'flux_aper90_ave_m', 'flux_aper90_ave_h','flux_aper90_ave_b']:
    print(df_com.loc[df_com[col+'_x']!=df_com[col+'_y'], ['name','dof_x','dof_y',col+'_x','e_'+col+'_x',col+'_y','e_'+col+'_y']])

3003
                       name  dof_x  dof_y  flux_aper90_ave_s_x  \
32    2CXO J003536.8-433615    1.0    0.0         8.125549e-18   
129   2CXO J030514.1+034853    1.0    0.0         7.998773e-18   
155   2CXO J033206.9-275610   26.0   24.0         1.139916e-17   
156   2CXO J033207.1-275129   82.0   40.0         8.515793e-18   
159   2CXO J033212.9-275237   82.0   73.0         1.079743e-17   
...                     ...    ...    ...                  ...   
2798  2CXO J054609.6-000331   13.0   12.0         9.325128e-18   
2799  2CXO J054612.2-000807   13.0    2.0         8.730896e-18   
2801  2CXO J054613.5-000855   13.0   -1.0         7.978846e-18   
2809  2CXO J054633.3+000251    1.0    0.0         8.443753e-18   
2951  2CXO J104606.1-595809    1.0    0.0         8.057672e-18   

      e_flux_aper90_ave_s_x  flux_aper90_ave_s_y  e_flux_aper90_ave_s_y  
32             6.027031e-18         4.205000e-16           3.196000e-16  
129            6.028084e-18         3.186751e-15      

In [58]:
#df_ave = pd.read_csv(f'{data_dir}/{field_name}_ave.csv')
#df_ave_pre = pd.read_csv(f'{data_dir}/CSCv2_TD_ave.csv')

df.columns[:50]

Index(['name', 'usrid', 'ra', 'dec', 'err_ellipse_r0', 'err_ellipse_r1',
       'err_ellipse_ang', 'significance', 'extent_flag', 'pileup_flag',
       'sat_src_flag', 'streak_src_flag', 'conf_flag', 'flux_aper90_mean_b',
       'e_flux_aper90_mean_b', 'flux_aper90_mean_h', 'e_flux_aper90_mean_h',
       'flux_aper90_mean_m', 'e_flux_aper90_mean_m', 'flux_aper90_mean_s',
       'e_flux_aper90_mean_s', 'kp_intra_prob_b', 'ks_intra_prob_b',
       'var_inter_prob_b', 'nh_gal', 'flux_powlaw_mean', 'e_flux_powlaw_mean',
       'powlaw_gamma_mean', 'e_powlaw_gamma_mean', 'powlaw_nh_mean',
       'e_powlaw_nh_mean', 'powlaw_ampl_mean', 'e_powlaw_ampl_mean',
       'powlaw_stat', 'remove_code', 'flux_significance_b_max',
       'flux_aper90_ave_s', 'e_flux_aper90_ave_s', 'flux_aper90_ave_m',
       'e_flux_aper90_ave_m', 'flux_aper90_ave_h', 'e_flux_aper90_ave_h',
       'chisqr', 'dof', 'kp_prob_b_max', 'var_inter_prob', 'significance_max',
       'flux_flag', 'flux_aper90_ave_b', 'e_flux_ap

In [47]:
df_com = pd.merge(df_ave[['name','ra','dec','dof','Class','flux_aper90_ave_s', 'e_flux_aper90_ave_s', 'flux_aper90_ave_m','e_flux_aper90_ave_m', 'flux_aper90_ave_h', 'e_flux_aper90_ave_h','flux_aper90_ave_b','e_flux_aper90_ave_b']],df_ave_pre[['name','Class','dof','flux_aper90_ave_s', 'e_flux_aper90_ave_s', 'flux_aper90_ave_m','e_flux_aper90_ave_m', 'flux_aper90_ave_h', 'e_flux_aper90_ave_h','flux_aper90_ave_b','e_flux_aper90_ave_b']],on='name',how='outer')
print(len(df_com))

for col in ['flux_aper90_ave_s',  'flux_aper90_ave_m', 'flux_aper90_ave_h']:
    print(df_com.loc[df_com[col+'_x']!=df_com[col+'_y'], ['name','dof_x','dof_y',col+'_x','e_'+col+'_x',col+'_y','e_'+col+'_y']])

3012
                       name  dof_x  dof_y  flux_aper90_ave_s_x  \
32    2CXO J003536.8-433615    1.0    0.0         8.125549e-18   
129   2CXO J030514.1+034853    1.0    0.0         7.998773e-18   
155   2CXO J033206.9-275610   26.0   24.0         1.139916e-17   
156   2CXO J033207.1-275129   82.0   40.0         8.515793e-18   
159   2CXO J033212.9-275237   82.0   73.0         1.079743e-17   
...                     ...    ...    ...                  ...   
2807  2CXO J054609.6-000331   13.0   12.0         9.325128e-18   
2808  2CXO J054612.2-000807   13.0    2.0         8.730896e-18   
2810  2CXO J054613.5-000855   13.0   -1.0         7.978846e-18   
2818  2CXO J054633.3+000251    1.0    0.0         8.443753e-18   
2960  2CXO J104606.1-595809    1.0    0.0         8.057672e-18   

      e_flux_aper90_ave_s_x  flux_aper90_ave_s_y  e_flux_aper90_ave_s_y  
32             6.027031e-18         4.205000e-16           3.196000e-16  
129            6.028084e-18         3.186751e-15      

In [16]:
df_final.head()

,name,ra,dec,PU,significance,Fcsc_s,e_Fcsc_s,Fcsc_m,e_Fcsc_m,Fcsc_h,...,W4mag,e_W4mag,cp_flag_allwise,Class,cp_flag_wise12,which_wise12,W1mag,e_W1mag,W2mag,e_W2mag
0,2CXO J000009.3+135618,0.039125,13.938494,0.79,1.95,1.671074e-15,1.098049e-15,4.844000e-16,4.844000e-16,1.989924e-15,...,8.827,0.462,0,AGN,0.0,allwise,15.489,0.045,14.605,0.059
1,2CXO J000230.7+004959,0.627958,0.833072,0.72,11.07,5.675297e-14,6.911736e-15,2.874798e-14,5.285001e-15,5.735159e-14,...,7.915,0.235,0,AGN,0.0,allwise,14.497,0.030,13.178,0.031
2,2CXO J000622.6-000424,1.594333,-0.073572,0.78,25.42,1.544000e-13,1.220000e-14,1.165798e-13,7.650074e-15,4.038000e-13,...,8.398,NaN,0,AGN,0.0,allwise,15.299,0.041,14.245,0.049
3,2CXO J000659.2-001740,1.747042,-0.294661,0.84,4.11,1.069845e-14,3.658216e-15,6.211934e-15,2.773510e-15,1.372718e-14,...,NaN,NaN,-8,AGN,0.0,catwise,16.549,0.037,15.650,0.057
4,2CXO J000703.6+155423,1.765000,15.906575,0.72,8.63,4.660000e-15,2.824000e-15,1.593202e-14,5.625001e-15,5.350000e-13,...,5.208,0.039,0,AGN,0.0,allwise,11.610,0.023,10.588,0.021


In [17]:
df_current = pd.read_csv('/home/orion51/Desktop/Research/MUWCLASS/MUWCLASS_CSCv2/files/CSC_TD_MW_remove.csv')
df_f = prepare_cols(df_current, cp_thres=0, vphas=False,gaiadata=False,cp_conf_flag=False, TD=True, NS_MWdrop=False, STAR_classremove=['HM-STAR','LM-STAR','YSO'])
df_final = df_current[df_current['name'].isin(df_f['name'])]

Remove 21 [('LM-STAR', 1), ('YSO', 20)]
Final breakdown 2941 [('AGN', 1390), ('CV', 44), ('HM-STAR', 118), ('HMXB', 26), ('LM-STAR', 207), ('LMXB', 41), ('NS', 87), ('NS_BIN', 24), ('YSO', 1004)]


In [18]:
df_final['SubClass']

0          Q
1          Q
2          Q
3          Q
4          A
        ... 
2957     III
2958    I/II
2959      II
2960      II
2961      II
Name: SubClass, Length: 2941, dtype: object

In [19]:
df_star = df_final[df_final['Class']=='LM-STAR'].reset_index(drop=True)
df_star['Spectral_type'] = df_star.apply(lambda r: r.SubClass[0:2],axis=1)
print(len(df_star))
print(df_star['Spectral_type'].value_counts())

df_GK = df_star[df_star['Spectral_type']=='GK'].reset_index(drop=True)

GK = df_f[df_f.name.isin(df_GK.name)]

GK = GK.rename(columns={'PU':'PU_X','Fcsc_s':'F_s','Fcsc_m':'F_m','Fcsc_h':'F_h','Fcsc_b':'F_b',\
                            'Gmag':'G','BPmag':'BP','RPmag':'RP','Jmag':'J','Hmag':'H','Kmag':'K','W1mag':'W1','W2mag':'W2','W3mag':'W3',\
                            'var_intra_prob':'P_intra', 'var_inter_prob':'P_inter'})

GK.reset_index(drop=True).to_csv('./data/TD_GK.csv',index=False)

df_star['Spectral_type'] = df_star.apply(lambda r: r.SubClass[0],axis=1)
print(len(df_star))
print(df_star['Spectral_type'].value_counts())

for tp in ['G','K','M']:

    df_tp = df_star[df_star['Spectral_type']==tp].reset_index(drop=True)

    tpe = df_f[(df_f.name.isin(df_tp.name)) & ~(df_f.name.isin(GK.name))].reset_index(drop=True)

    tpe = tpe.rename(columns={'PU':'PU_X','Fcsc_s':'F_s','Fcsc_m':'F_m','Fcsc_h':'F_h','Fcsc_b':'F_b',\
                            'Gmag':'G','BPmag':'BP','RPmag':'RP','Jmag':'J','Hmag':'H','Kmag':'K','W1mag':'W1','W2mag':'W2','W3mag':'W3',\
                            'var_intra_prob':'P_intra', 'var_inter_prob':'P_inter'})
    print(len(tpe))
    
    tpe.to_csv(f'./data/TD_{tp}.csv',index=False)



207
GK    21
F8    17
F5    17
Md    13
G0    13
Fd    12
A2    11
A0     9
G5     9
K0     9
F2     9
F0     5
K1     4
G8     4
F6     4
F4     3
M      3
M5     3
F      3
A1     2
G6     2
A3     2
K2     2
A6     2
G9     2
F7     2
F3     2
G3     2
M3     2
F9     2
M4     1
A8     1
K      1
G1     1
K3     1
A7     1
M1     1
A4     1
A      1
G7     1
K5     1
F1     1
M7     1
A5     1
M0     1
G2     1
Name: Spectral_type, dtype: int64
207
F    77
G    56
A    31
M    25
K    18
Name: Spectral_type, dtype: int64
35
18
25


In [ ]:
df_M = df_star[df_star['Spectral_type']=='M'].reset_index(drop=True)

df_Md = df_f[df_f.name.isin(df_M.name)]

In [5]:
df_star = df_final[df_final['Class']=='LM-STAR'].reset_index(drop=True)
df_star['Spectral_type'] = df_star.apply(lambda r: r.SubClass[0:2],axis=1)
print(len(df_star))
print(df_star['Spectral_type'].value_counts())
df_M = df_star[df_star['Spectral_type']=='M'].reset_index(drop=True)
df_apoge = df_star[df_star['ref']=='2020AJ....160..120J'].reset_index(drop=True)
df_spec = df_star[df_star['ref']!='2020AJ....160..120J'].reset_index(drop=True)
print(len(df_apoge),len(df_spec))
print(df_apoge['SubClass'].value_counts())
print(df_spec['SubClass'].value_counts())
#df_spec['Spectral_type'] = df_spec.apply(lambda r: r.SubClass[0],axis=1)
print(df_spec['Spectral_type'].value_counts())
#'2020AJ....160..120J'

207
GK    21
F8    17
F5    17
Md    13
G0    13
Fd    12
A2    11
A0     9
G5     9
K0     9
F2     9
F0     5
K1     4
G8     4
F6     4
F4     3
M      3
M5     3
F      3
A1     2
G6     2
A3     2
K2     2
A6     2
G9     2
F7     2
F3     2
G3     2
M3     2
F9     2
M4     1
A8     1
K      1
G1     1
K3     1
A7     1
M1     1
A4     1
A      1
G7     1
K5     1
F1     1
M7     1
A5     1
M0     1
G2     1
Name: Spectral_type, dtype: int64
46 161
GKd_b    8
GKd_c    6
Md_a     5
Fd_b     5
Fd_a     4
GKd_a    4
Md_c     3
Md_d     3
Fd_c     2
Md_b     2
GKd_d    1
GKg_a    1
Fd_d     1
GKg_c    1
Name: SubClass, dtype: int64
F8       14
F5       12
K0        8
F2        8
A2        8
         ..
F3        1
F5/7      1
K5        1
F6/7V     1
K1        1
Name: SubClass, Length: 75, dtype: int64
F5    17
F8    17
G0    13
A2    11
A0     9
G5     9
K0     9
F2     9
F0     5
K1     4
G8     4
F6     4
M      3
F4     3
F      3
M5     3
K2     2
A3     2
F7     2
A6     2
G9   

In [23]:
df_Md = df_f[df_f.name.isin(df_M.name)]
df_Md.columns[:40]

Index(['name', 'ra', 'dec', 'PU', 'significance', 'Fcsc_s', 'e_Fcsc_s',
       'Fcsc_m', 'e_Fcsc_m', 'Fcsc_h', 'e_Fcsc_h', 'flux_aper90_ave_b',
       'e_flux_aper90_ave_b', 'var_intra_prob', 'var_inter_prob', 'CSC_flags',
       'EDR3Name_gaia', 'RA_pmcor_gaia', 'DEC_pmcor_gaia', 'Gmag', 'e_Gmag',
       'BPmag', 'e_BPmag', 'RPmag', 'e_RPmag', 'Plx_gaia', 'e_Plx_gaia',
       'PM_gaia', 'rgeo', 'b_rgeo', 'B_rgeo', 'rpgeo', 'b_rpgeo', 'B_rpgeo',
       'cp_flag_gaia', '_2MASS_2mass', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag'],
      dtype='object')

In [24]:
df_Md = df_Md.rename(columns={'PU':'PU_X','Fcsc_s':'F_s','Fcsc_m':'F_m','Fcsc_h':'F_h','Fcsc_b':'F_b',\
                            'Gmag':'G','BPmag':'BP','RPmag':'RP','Jmag':'J','Hmag':'H','Kmag':'K','W1mag':'W1','W2mag':'W2','W3mag':'W3',\
                            'var_intra_prob':'P_intra', 'var_inter_prob':'P_inter'})
# add broad band flux
#TD[]
df_Md.reset_index(drop=True).to_csv('./data/TD_Mdwarf.csv',index=False)